In [ ]:
!pip install flask pyngrok diffusers transformers accelerate --quiet
!ngrok config add-authtoken Your Authtoken

from flask import Flask, request, jsonify
from threading import Thread
from pyngrok import ngrok
import base64
from io import BytesIO
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image


pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16
).to("cuda")


app = Flask(__name__)

@app.route("/")
def home():
    return "✅ Colab backend is running!"

@app.route("/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prompt = data.get("prompt", "").strip()

    if not prompt:
        return jsonify({"error": "No prompt provided"}), 400

    try:
        image = pipe(prompt).images[0]
        buf = BytesIO()
        image.save(buf, format="PNG")
        encoded = base64.b64encode(buf.getvalue()).decode("utf-8")
        return jsonify({"image_base64": encoded})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


def run():
    app.run()

Thread(target=run).start()


public_url = ngrok.connect(5000)
print("🚀 Your backend is live at:", public_url)
